# Konya’daki Ev Fiyatlarını ve Özelliklerini Web Scraping ile Çekme

In [1]:
!pip install selenium

In [2]:
from selenium.webdriver import Chrome
from bs4 import BeautifulSoup
import pandas as pd

def extract_adverts(page_url):
    driver = Chrome()
    driver.get(page_url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    return [link.get('href') for link in soup.select('li.zl-card a.zl-card-inner')]

def extract_data(ad_url):
    driver = Chrome()
    driver.get(ad_url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    price = soup.find('strong', itemprop='price').text.strip()
    location = soup.find('h2').text.strip()
    features = {li.find('strong').text.strip(): li.find('span').text.strip() for li in soup.find_all('li', class_='col-md-6')}
    features.update({"Location": location, "Price": price})
    return features


base_url = "https://www.zingat.com/konya-satilik-konut?page={}"
all_urls = sum([extract_adverts(base_url.format(i)) for i in range(1,15)], []) 
df = pd.DataFrame([extract_data(f"https://www.zingat.com{url}") for url in all_urls])
df.to_csv("konya_satilik_konutlar.csv", index=False)

In [3]:
df.to_csv("konya_satilik_konutlar.csv", index=False)

In [4]:
df.head()

,İlan Tarihi,İlan no,Net m²,Brüt m²,Oda Sayısı,Banyo Sayısı,Binadaki Kat Sayısı,Isıtma Tipi,Konut Şekli,Bulunduğu Kat,...,Price,Görüntülü Ev Gezintisi\n?,Hayvan Dostu Patili Evler,Mobilya Durumu,Takasa Uygun,Aidat,Tapu Durumu,Ada,Parsel,Yapı Tipi
0,30 Eylül 2024,5145472,115,128m²,3+1,1,3,Kombi (Doğalgaz),Ara Kat,2,...,1.860.000 TL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21 Eylül 2024,5115610,130,141m²,2+1,1,5,Kombi (Doğalgaz),NaN,3,...,1.675.000 TL,?,Evet,Eşyasız,NaN,NaN,NaN,NaN,NaN,NaN
2,26 Eylül 2024,5142613,155,180m²,4+1,2,5,NaN,Ara Kat,2,...,3.500.000 TL,NaN,NaN,Eşyasız,Evet,0 TL (aylık),NaN,NaN,NaN,NaN
3,20 Eylül 2024,5055209,130,145m²,2+1,1,NaN,Kalorifer (Doğalgaz),Bahçe Dubleksi,1,...,1.625.000.000 TL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18 Eylül 2024,5137354,145,165m²,3+1,1,5,Kombi (Doğalgaz),Ara Kat,4,...,3.550.000 TL,NaN,NaN,Eşyasız,NaN,0 TL (aylık),NaN,NaN,NaN,NaN
